In [ ]:
import tensorflow as tf
from tensorflow.keras.models import sequential
from tensorflow.keras.layers import Embedding,LSTM, control_dependencies
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequence
import numpy as np
import regex as re

In [ ]:
#path for the file
file_path="pizza.txt"

#reading the file
with open (file_path,"r") as file:
    text=file.read()

    #spliting the data input according to the regex provided
    data=[sentence.strip() for sentence in re.split(r'(?<=[.!?]\s+)',text) if sentence.strip()]

